In [2]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import ctypes
from ctypes import byref, c_void_p, c_int, c_size_t, c_int64, c_uint, c_uint8, c_uint16

In [2]:
lib = ctypes.cdll.LoadLibrary(r"C:\Program Files\Point Grey Research\Spinnaker\bin64\vs2015\SpinnakerC_v140.dll")

In [3]:
hSystem = ctypes.c_void_p()

err = lib.spinSystemGetInstance(ctypes.byref(hSystem))
hSystem

c_void_p(9300848L)

In [4]:
hCameraList = c_void_p()
err = lib.spinCameraListCreateEmpty(byref(hCameraList))
print(err, hCameraList)
err = lib.spinSystemGetCameras(hSystem, hCameraList)
print(err, hCameraList)


(0, c_void_p(9469248L))
(0, c_void_p(9469248L))


In [5]:
# Retrieve number of cameras
numCameras = c_size_t(0)

err = lib.spinCameraListGetSize(hCameraList, byref(numCameras))
print(err, numCameras.value)

(0, 1L)


In [6]:
#spinCamera hCamera = NULL;
hCamera = c_void_p()

err = lib.spinCameraListGet(hCameraList, 0, byref(hCamera))
print(err, hCamera)


(0, c_void_p(191022672L))


In [7]:
hNodeMapTLDevice = c_void_p()
err = lib.spinCameraGetTLDeviceNodeMap(hCamera, byref(hNodeMapTLDevice))
print(err, hNodeMapTLDevice)

(0, c_void_p(9844976L))


In [8]:
err = lib.spinCameraInit(hCamera)
print(err)

0


In [9]:
hNodeMap = c_void_p()

err = lib.spinCameraGetNodeMap(hCamera, byref(hNodeMap))
print(err, hNodeMap)

(0, c_void_p(213094976L))


In [10]:
def get_enum_int_by_name(hEnumNode, name):
    hEnumEntry = c_void_p()
    enumInt = c_int64()
    err = lib.spinEnumerationGetEntryByName(hEnumNode, name, byref(hEnumEntry))
    print(err)
    assert err == 0
    err = lib.spinEnumerationEntryGetIntValue(hEnumEntry, byref(enumInt))
    assert err == 0
    return enumInt.value

In [11]:
hAcquisitionMode = c_void_p()
err = lib.spinNodeMapGetNode(hNodeMap, b"AcquisitionMode", byref(hAcquisitionMode))
print(err, hAcquisitionMode)
err = lib.spinEnumerationSetIntValue(hAcquisitionMode, get_enum_int_by_name(hAcquisitionMode, b'Continuous'))
print(err)

(0, c_void_p(213094720L))
0
0


In [12]:
err = lib.spinCameraBeginAcquisition(hCamera)
print(err)

0


In [13]:
hResultImage = c_void_p()
err = lib.spinCameraGetNextImage(hCamera, byref(hResultImage))
print(err, hResultImage)
isIncomplete = ctypes.c_bool(False)
err = lib.spinImageIsIncomplete(hResultImage, byref(isIncomplete))
print(err, isIncomplete)


(0, c_void_p(192933520L))
(0, c_bool(True))


In [14]:
width = ctypes.c_uint(0)
height = ctypes.c_uint(0)

err = lib.spinImageGetWidth(hResultImage,byref(width) )
print(err, width.value,)
err = lib.spinImageGetHeight(hResultImage,byref(height) )
print(err, height.value,)

(0, 2109765568L)
(0, 2067822400L)


In [18]:
hConvertedImage = c_void_p()
err = lib.spinImageCreateEmpty(byref(hConvertedImage))
print(err, hConvertedImage)
err = lib.spinImageConvert(hResultImage, 0, hConvertedImage)
print(err)


(0, c_void_p(192931920L))
-1001


In [19]:
err = lib.spinImageSave(hConvertedImage, b"test.jpg", -1)
print(err)

-1002


In [ ]:
asdf = ctypes.c_uint64()
ctypes.sizeof(asdf)

In [ ]:
lib.spinImageGetData.argtypes = (c_void_p,ctypes.POINTER(ctypes.POINTER(c_uint8)))
lib.spinImageGetData.restype = c_uint8
ppData = ctypes.POINTER(c_uint8)()
pSize = c_uint(0)
print(lib.spinImageGetData(hConvertedImage, ppData))
print(lib.spinImageGetBufferSize(hConvertedImage, byref(pSize)))

In [ ]:
data = np.ctypeslib.as_array(ppData,(1920,1200))

In [ ]:
data

In [ ]:
X,Y = np.meshgrid(np.arange(0,1200), np.arange(0,1920))
plt.pcolormesh(X,Y,data, cmap='gray')
plt.colorbar()

plt.axis('tight')
plt.axis('equal')

In [ ]:
err = lib.spinImageDestroy(hConvertedImage)
print(err)

In [ ]:
err = lib.spinImageRelease(hResultImage)
print(err)

In [ ]:
err = lib.spinCameraEndAcquisition(hCamera)
print(err)

In [ ]:
err = lib.spinCameraRelease(hCamera)
print(err)

In [ ]:
err = lib.spinCameraListClear(hCameraList)
print(err)

In [ ]:
err = lib.spinCameraListDestroy(hCameraList)
print(err)

In [ ]:
err = lib.spinSystemReleaseInstance(hSystem)
print(err)

In [ ]:
import numpy as np
np.zeros(5)

In [ ]:
import sys
print(sys.path)

In [ ]:
['C:\\Users\\lab\\Documents\\foundry_scope\\ScopeFoundryHW\\flircam', 'C:\\Users\\lab\\Documents\\foundry_scope', 'C:\\Users\\lab\\Anaconda3\\envs\\scopefoundry\\DLLs', 'C:\\Users\\lab\\Anaconda3\\envs\\scopefoundry\\lib', 'C:\\Users\\lab\\Anaconda3\\envs\\scopefoundry', 'C:\\Users\\lab\\Anaconda3\\envs\\scopefoundry\\lib\\site-packages', 'C:\\Users\\lab\\Anaconda3\\envs\\scopefoundry\\lib\\site-packages\\win32', 'C:\\Users\\lab\\Anaconda3\\envs\\scopefoundry\\lib\\site-packages\\win32\\lib', 'C:\\Users\\lab\\Anaconda3\\envs\\scopefoundry\\lib\\site-packages\\Pythonwin', 'C:\\Users\\lab\\Anaconda3\\envs\\scopefoundry\\python37.zip']


In [4]:
asdf = np.array([2,3,4], dtype=int)
typecodes = np.ctypeslib._get_typecodes()
typecodes[asdf.__array_interface__['typestr']]

AttributeError: module 'numpy.ctypeslib' has no attribute '_get_typecodes'